# Process the data

## An example to split item talk pages into threads, posts, words

### 1. Split into threads and posts

In [ ]:
import csv
import sys
import codecs
import os
import re
from difflib import SequenceMatcher
import itertools
import json
import codecs
import pandas as pd


def UTC_separation(thread):
    posts_lst=[]
    posts=re.split(r'(\d\d:\d\d, \d+ \w+ \d\d\d\d \(UTC\))',thread)# split the posts based on the timestamp
    for i in range(0,len(posts)-1, 2):
        posts_lst.append(posts[i]+posts[i+1])# connects post with timestamp because the splot separates them
    
    return posts_lst

#to separate discussions with no title
def thread_without_title(temp, page_title, save_path_data):
    if ((temp[0:2]=='{{') and (('documentation' in temp) or ('Documentation' in temp))) or ((temp[0:2]=='{{') and (('Interwiki conflict' in temp) or ('interwiki conflict' in temp))) or ('#REDIRECT' in temp): # checks if the text includes only documentation, not discussions
         trash=pd.DataFrame({'page_title':[page_title], 'text':temp})
         trash.to_csv(str(save_path_trash),mode='a',index=False, header=not os.path.exists(str(save_path_trash)))
    else:
        thread_title='No subject'
        posts_lst = UTC_separation(temp)
        #store posts with the corresponding sibject
        df=pd.DataFrame({'page_title':[page_title]*len(posts_lst), 'thread_title':[thread_title]*len(posts_lst),'post':posts_lst})
        #save to csv 
        df.to_csv(str(save_path_data), mode='a',index=False, header=not os.path.exists(str(save_path_data)))


def thread_with_title(titles, temp, page_title, save_path_data):
    for j in range(len(temp)):
        if (temp[j] in titles):
            if (j+1)==(len(temp)):
                thread_title=temp[j]
                posts_lst = []
                    #store posts with the corresponding sibject
                df=pd.DataFrame({'page_title':[page_title]*len(posts_lst), 'thread_title':[thread_title]*len(posts_lst),'post':posts_lst})
                    #save to csv 
                df.to_csv(str(save_path_data), mode='a',index=False, header=not os.path.exists(str(save_path_data)))
            else:
                thread_title=temp[j]
                posts_lst = UTC_separation(temp[j+1])
                    #store posts with the corresponding sibject
                df=pd.DataFrame({'page_title':[page_title]*len(posts_lst), 'thread_title':[thread_title]*len(posts_lst),'post':posts_lst})
                    #save to csv 
                df.to_csv(str(save_path_data), mode='a',index=False, header=not os.path.exists(str(save_path_data)))
        
    

#function to create a file with the separated posts  
def separate_discussions(text, page_title, save_path_data):

    titles = re.findall('==(.*)==', text)# find the titles in the discussion  
    titles=[s.replace('=', '') for s in titles]# removes unecessery = in the title
    
    
    #titles=list(set(titles)) 
    temp= text.split('==') # split the text based on == , includided in the title
    temp = [x for x in temp if x != ''] # stores the splited parts
    temp=[s.strip('=') for s in temp]# removes unecessery = in the title

    # ---Step 1: there is no title in the page
    if titles==[]: # check if there is title in the begining of the page
        thread_without_title(temp[0], page_title, save_path_data) 
    else:
        if temp[0] not in titles:
            thread_without_title(temp[0], page_title, save_path_data)# ---Step 2: the first thread do not have title 
            thread_with_title(titles, temp, page_title, save_path_data)# ignores the first if does not have a title and process the rest threads
        else:
            thread_with_title(titles, temp, page_title, save_path_data)# ---Step 3: a;; threads have titles 



#this is to extent the size of the reading csv cell
# csv.field_size_limit(sys.maxsize)
file_path='data/all_item_pages/'
save_path_trash='trash_files.csv'
save_path_data='item_talk_pages.csv' # path to save posts and threads

# read the files in a diectory and process
for file in os.listdir(str(file_path)):
    data=pd.read_csv(str(file_path)+str(file), encoding='utf-8')# read file
    for row in range(len(data)):# read row by row (i.e., page by page)
        print(row)
        page_title=data.loc[row,'page_title']
        if isinstance(data.loc[row,'text'],str):# check of the row is nan. If it is nan the type is float not str
            separate_discussions(data.loc[row,'text'], page_title, save_path_data)
        else:
            empty=pd.DataFrame({'page_title':[page_title], 'text':'empty'})
            empty.to_csv(str(save_path_trash),mode='a',index=False, header=not os.path.exists(str(save_path_trash)))


### 2. Count words

In [ ]:
import pandas as pd
from utils import preprocess_sentence

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# clean and then tokenize the posts
def tokenize_post(post):
    text=preprocess_sentence(post)
    word_list=text.split()
    #num_words_text=len(word_list)

    return word_list


def post_level_stats(PATH_input, PATH_output):
    data = pd.read_csv(str(PATH_input), header=None, encoding='utf-8')
    

    new_data = pd.DataFrame(columns=['token_post','num_words', 'thread_subject', 'discussion_page_name'])#create a df to save the info


    for row in range(len(data)):
        print(row, len(data))
        row_post=data.loc[row,'post']
        token_post=tokenize_post(row_post)
        
        save_row={'token_post':token_post,'num_words':len(token_post),'thread_subject':data.loc[row,'thread_subject'],'discussion_page_name':data.loc[row,'discussion_page_name'],'fixed_name':data.loc[row,'fixed_name'],'timestamp':data.loc[row,'timestamp']}
        new_data=new_data.append(save_row,ignore_index=True)


    new_data.to_csv(str(PATH_output), encoding='utf-8', index=False,  mode='a', header=False)

In [ ]:
PATH_input='item_talk_pages.csv'
PATH_output='stats_items.csv'

post_level_stats(PATH_input , PATH_output)
